# Import and Export of Optimization Problems

This notebook demonstrates how an optimization problem setup in **MASSpy** can be exported for use with other optimization solvers. This notebook is based on the [Optlang API documentation](https://optlang.readthedocs.io/en/latest/) and the COBRApy FAQ [How do I generate an LP file from a COBRA model?](https://cobrapy.readthedocs.io/en/latest/faq.html#How-do-I-generate-an-LP-file-from-a-COBRA-model?) 

<div class="alert alert-info"> 
Variables, constraints, objectives, and a name (if provided) are imported/exported through this method; however, solver configuration options are not.
</div>


In [1]:
try:
    import simplejson as json
except ImportError:
    import json
    
import cobra
from optlang import Model as OptModel

import mass.example_data
# Print list of available solvers
print(list(cobra.util.solver.solvers))

['glpk_exact', 'glpk', 'gurobi', 'scipy']


## Using Optlang


To facilitate the formation of the mathematical optimization problem, **MASSpy** utilizes the [Optlang python package](https://github.com/opencobra/optlang) <cite data-cite="JCS17">(Jensen et al., 2017)</cite>. As stated in the [documentation](https://optlang.readthedocs.io/en/latest/):

1. Optlang provides a common interface to a series of optimization tools, so different solver backends can be changed in a transparent way.
2. Optlang takes advantage of the symbolic math library [SymPy](https://www.sympy.org) to allow objective functions and constraints to be easily formulated from symbolic expressions of variables. 
3. Optlang interfaces with all solvers through importable python modules ([read more here](https://optlang.readthedocs.io/en/latest/installation.html#solvers)).


The following optimization solvers are supported:

* [GLPK](http://www.gnu.org/software/glpk/) (LP/MILP; via [swiglpk](https://github.com/biosustain/swiglpk))
* [CPLEX](https://www.ibm.com/products/ilog-cplex-optimization-studio) (LP/MILP/QP)
* [Gurobi](https://www.gurobi.com/) (LP/MILP/QP)

However, there are times where it would be preferrable to utilize other solvers and/or change programming environments in the process of setting up and performing optimizations. Fortunately, **Optlang** provides class methods for importing and exporting the optimization problem in both LP and JSON-compatible formats. The examples below demonstrate how the JSON format is utilized with **MASSpy** objects to facilitate the transference of optimization problems.

<div class="alert alert-warning"> 

It is generally NOT recommended to import optimization problems directly into the solvers, as the corresponding <strong>MASSpy</strong> objects are bypassed and therefore do not have any values updated to match the new state of the solver.

</div>

### Importing and Exporting with LP files
LP formulations of models can be used in conjunction with **Optlang** facilitate the exchange of optimization problems. 
Note the following:

1. Importing and exporting using LP formulations can change variable and constraint identifiers
2. LP formulations **do not** work with the ``scipy`` solver interface.

In [2]:
# Start with a fresh model
model = mass.example_data.create_example_model("textbook")
# Change the bounds for demonstration purposes
model.variables.HEX1.lb, model.variables.HEX1.ub = (-123, 456)
print(model.variables["HEX1"])

Set parameter Username
-123 <= HEX1 <= 456


### Exporting an LP file
For all solver interfaces in **Optlang**, the ``str`` representation of an ``optlang.interface.Model`` is the LP formulation of the problem.

To export the optimization problem into a file:

In [3]:
with open("problem.lp", "w") as file:
    file.write(str(model.solver))

Alternatively, the ``optlang.interface.Model.to_lp()`` method can be used, but note that variable and constraint identifiers may be changed. 

### Importing an LP file
The ``optlang.interface.Model.from_lp()`` method can be used to import an LP formulation of an optimization problem.

In [4]:
# Use new model to demonstrate how bounds change
model = mass.example_data.create_example_model("textbook")
print("Before: " + str(model.variables["HEX1"]))

# Load problem from JSON file
with open("problem.lp") as file:
    model._solver = OptModel.from_lp(file.read())
print("After: " + str(model.variables["HEX1"]))

Before: 0 <= HEX1 <= 1000.0
After: -123.0 <= HEX1 <= 456.0


### Importing and Exporting with JSON files

In [5]:
# Start with a fresh model
model = mass.example_data.create_example_model("textbook")
# Change the bounds for demonstration purposes
model.variables.HEX1.lb, model.variables.HEX1.ub = (-654, 321)
print(model.variables.HEX1)

-654 <= HEX1 <= 321


#### Exporting using JSON

Problems formulated in **Optlang** can be exported using the [optlang.interface.Model.to_json](https://optlang.readthedocs.io/en/latest/Model.html#optlang.interface.Model.to_json) class method. First, the ``to_json`` class method exports a JSON compatible ``dict`` containing the variables, constraints, objectives, and an optional name from the ``optlang.interface.Model``. The ``dict`` is then passed to [json.dump](https://docs.python.org/3.7/library/json.html#json.dump) to save the optimization problem as a JSON file.

To export the optimization problem into a file:

In [6]:
with open("problem.json", "w") as file:
    json.dump(model.solver.to_json(), file)

#### Importing using JSON
Problems can be imported into **Optlang** using the [optlang.interface.Model.from_json](https://optlang.readthedocs.io/en/latest/Model.html#optlang.interface.Model.from_json) class method. First, a JSON compatible ``dict`` is loaded from a file using the [json.load](https://docs.python.org/3.7/library/json.html#json.load). The ``dict`` is then passed to the ``from_json`` class method to load the variables, constraints, objectives, and an optional name into the ``optlang.interface.Model`` (imported as "OptModel" in this example").

To import the optimization problem from a file:

In [7]:
# Use new model to demonstrate how bounds change
model = mass.example_data.create_example_model("textbook")
print("Before: " + str(model.variables.HEX1))

# Load problem from JSON file
with open("problem.json") as file:
    model._solver = OptModel.from_json(json.load(file))
print("After: " + str(model.variables.HEX1))

Before: 0 <= HEX1 <= 1000.0
After: -654 <= HEX1 <= 321


### Adding a solver interface

For an optimization solver that does not currently have an interface, consider [adding a solver interface](https://optlang.readthedocs.io/en/stable/developers.html#add-solver-interface). 